<a href="https://colab.research.google.com/github/Deeks900/GenAIBootcamp/blob/main/5_NewsClassification_MLP_trial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Apr 24 07:01:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# !pip install --upgrade  textblob gensim pytorch-nlp swifter
!pip install --upgrade  textblob gensim pytorch-nlp swifter 'numpy<1.26'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 112.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.


In [1]:
import multiprocessing
import sys

import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob

embedding_dim = 100
epochs=10
batch_size = 250
corpus_size=5000

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  random.seed(42)


set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
%%writefile get_data.sh
if [ ! -f news.csv ]; then
  wget -O news.csv https://www.dropbox.com/scl/fi/7p0lipgmk5b1rf6wwmb9m/news.csv?rlkey=i3u8u5n432kdxob7txco82dht&st=212yyytp&dl=0
fi

Writing get_data.sh


In [3]:
!bash get_data.sh

--2025-04-24 07:04:30--  https://www.dropbox.com/scl/fi/7p0lipgmk5b1rf6wwmb9m/news.csv?rlkey=i3u8u5n432kdxob7txco82dht
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc344dc4a55e74fe5634fe032c89.dl.dropboxusercontent.com/cd/0/inline/CobVpDlz_PK6PMOP0FcKoVNGSUH5oiTEAw8PCy3YKUda4UZLoGpQtOynahJtFM-vXIhcJckFlVjUpUEJibGJUhGrZudEWTlrjEFVnGAjifL93PY8GfXBmfW3P6De-MbFvArLcZu76AHn_cbt6zDwLcxt/file# [following]
--2025-04-24 07:04:31--  https://uc344dc4a55e74fe5634fe032c89.dl.dropboxusercontent.com/cd/0/inline/CobVpDlz_PK6PMOP0FcKoVNGSUH5oiTEAw8PCy3YKUda4UZLoGpQtOynahJtFM-vXIhcJckFlVjUpUEJibGJUhGrZudEWTlrjEFVnGAjifL93PY8GfXBmfW3P6De-MbFvArLcZu76AHn_cbt6zDwLcxt/file
Resolving uc344dc4a55e74fe5634fe032c89.dl.dropboxusercontent.com (uc344dc4a55e74fe5634fe032c89.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6

In [5]:
path = './news.csv'
news_pre = pd.read_csv(path, header=0).sample(n=corpus_size).reset_index(drop=True)
news_pre

,category,title
0,Sports,"It #39;s been swell, Pedro"
1,World,US soldiers flock to laser eye clinic
2,Business,"MSU med school plan to move has flaws, study says"
3,Business,Viacom in China Tie-Up with Beijing TV (Reuters)
4,Sports,Coulthard has one race to prove his worth
...,...,...
4995,World,"NGOs working to topple regime, says Mugabe"
4996,Sci/Tech,IBM's PC unit lost money from 2001 onwards
4997,World,Into the abyss
4998,Sports,Lucchino thinks it was a bad move


In [11]:
def preprocess_text(text, should_join=True):
    # Use the tokenizer to tokenize into words, lowercase them, remove punctuation, and finally use gensim.utils.simple_preprocess(text)
    # text = None
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [14]:
import swifter
# Use swifter to apply the preprocessin and save that pandas series to a file news_processed.csv
news_pre['title']
# news = news_pre['title'].swifter.apply(preprocess_text)
news = news_pre['title'].swifter.apply(preprocess_text)
news.to_csv('news_processed.csv', index=False, header=False)
news


Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

,title
0,it been swell pedro
1,us soldiers flock to laser eye clinic
2,msu med school plan to move has flaws study says
3,viacom in china tie up with beijing tv reuters
4,coulthard has one race to prove his worth
...,...
4995,ngos working to topple regime says mugabe
4996,ibm pc unit lost money from onwards
4997,into the abyss
4998,lucchino thinks it was bad move


In [17]:
from gensim.models import Word2Vec
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'news_processed.csv'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield preprocess_text(line, should_join=False)

sentences = MyCorpus()
first_sentence = next(iter(sentences))
print(first_sentence)

# Get a word2vec model using gensim.models and passing the sentences using MyCorpus()
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=2)

['it', 'been', 'swell', 'pedro']


In [23]:
# weights = None  # Get the weights of the model (the embedding) and convert to tensor. Hint: Check word2vec_model.wv
# vocab_size = None  # get vocab size from index_to_key in word2vec_model.wv

vocab_size = len(model.wv.key_to_index)
print(vocab_size)
weights = model.wv.vectors
weights[0]

8114


array([-0.1480588 ,  0.19271593,  0.04863212,  0.05906339,  0.01492517,
       -0.33448604,  0.09071042,  0.43350324, -0.10323492, -0.12722056,
       -0.00547168, -0.30356947, -0.08290465,  0.13293342,  0.03140416,
       -0.1107148 ,  0.08207574, -0.06879014, -0.06503376, -0.3937664 ,
        0.12976591,  0.03483959,  0.22334681, -0.08335888,  0.00664758,
       -0.01451853, -0.16168949,  0.05352098, -0.22594592, -0.01606136,
        0.19991963, -0.02096082,  0.04484121, -0.22247857, -0.01678781,
        0.14135781,  0.15838116, -0.07832687, -0.07506708, -0.24480979,
       -0.00464079, -0.23295404, -0.16847771,  0.08516539,  0.19912796,
       -0.00918762, -0.17134292,  0.01675604,  0.10004286,  0.18651865,
        0.13146733, -0.18582559, -0.13146718, -0.05836325, -0.13552056,
       -0.0031904 ,  0.2000642 , -0.05891591, -0.24078608,  0.07671502,
       -0.03313306,  0.02866978, -0.01633516, -0.01215401, -0.1424844 ,
        0.25048018,  0.11034026,  0.15373237, -0.16016498,  0.27

In [22]:
weights.shape

(8114, 100)

In [24]:
news_preprocessed = pd.DataFrame()
news_preprocessed['label'] = news_pre.category.map({'Business': 0, 'Sports': 1, 'Sci/Tech': 2, 'World': 3})
news_preprocessed['title'] = news
news_preprocessed

,label,title
0,1,it been swell pedro
1,3,us soldiers flock to laser eye clinic
2,0,msu med school plan to move has flaws study says
3,0,viacom in china tie up with beijing tv reuters
4,1,coulthard has one race to prove his worth
...,...,...
4995,3,ngos working to topple regime says mugabe
4996,2,ibm pc unit lost money from onwards
4997,3,into the abyss
4998,1,lucchino thinks it was bad move


In [28]:
def textblob_tokenizer(text):
    return text.split()
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(textblob_tokenizer(row.title))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(news_preprocessed)   # Since 2 titles may have different number of words, we have to find the max length and fill with 0s if a title is shorter
maximum

17

In [30]:
X = np.zeros((len(news_preprocessed), maximum))   # Here we do what we said above
X
# Iterate through the news df and for every word, if it exists in the word2vec model, put into X for that review and that word the index of the embedding (check index_to_key)
# HINT: to iterate through a column of a pandas dataframe you do:

# for index, value in df.iterrows():
#      #do something

# FILL
# y = news_preprocessed.label

for i, row in news_preprocessed.iterrows():
    words = row.title.split()
    for j, word in enumerate(words[:maximum]):  # Limit to max length
        if word in model.wv.key_to_index:
            X[i, j] = model.wv.key_to_index[word]  # Assign word index

y = news_preprocessed.label
print(y)
print(X)

0       1
1       3
2       0
3       0
4       1
       ..
4995    3
4996    2
4997    3
4998    1
4999    2
Name: label, Length: 5000, dtype: int64
[[4.900e+01 1.378e+03 5.074e+03 ... 0.000e+00 0.000e+00 0.000e+00]
 [1.000e+01 3.680e+02 2.500e+03 ... 0.000e+00 0.000e+00 0.000e+00]
 [5.083e+03 5.084e+03 7.330e+02 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [6.100e+01 6.000e+00 6.112e+03 ... 0.000e+00 0.000e+00 0.000e+00]
 [8.113e+03 3.706e+03 4.900e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [7.200e+02 1.700e+01 2.521e+03 ... 0.000e+00 0.000e+00 0.000e+00]]


In [32]:
import torch.nn.functional as F
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert all datasets to tensors
# Convert to PyTorch tensors
X_train = torch.LongTensor(X_train)
X_test = torch.LongTensor(X_test)

y_train = F.one_hot(torch.tensor(y_train.values), num_classes=4).float()
y_test = F.one_hot(torch.tensor(y_test.values), num_classes=4).float()
# X_train = None
# X_test = None
# Convert y_train and y_test from an array of values between 0-3 to a one hot matrix tensor


In [33]:
class MeanLayer(nn.Module):

  def forward(self, x):
    return torch.mean(x, dim=1)

In [37]:
from torch.nn import Embedding, Linear, Sequential, LogSoftmax

# Create a sequential model like we have been doing, apply softmax.
weights = torch.tensor(weights)
weights.dtype
model  = Sequential(
    Embedding.from_pretrained(embeddings=weights, freeze=True, padding_idx=0),
    # add layers to get better re[presentation per word

    #pass from representation pof words (3 d) to rep of review (2d)

    MeanLayer(),

    # final part
    Linear(in_features=embedding_dim, out_features=4),
    LogSoftmax(dim=1)
)
model


Sequential(
  (0): Embedding(8114, 100, padding_idx=0)
  (1): MeanLayer()
  (2): Linear(in_features=100, out_features=4, bias=True)
  (3): LogSoftmax(dim=1)
)

In [38]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [42]:
def train_cbow(X, y, model, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        y_pred = model(X)

        # Calculate loss
        loss = loss_function(y_pred, y)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    # implement

In [43]:
trained_model = train_cbow(X_train, y_train, model, loss_function, optimizer, epochs=epochs)


Epoch [1/10], Loss: 1.3863
Epoch [2/10], Loss: 1.3863
Epoch [3/10], Loss: 1.3863
Epoch [4/10], Loss: 1.3863
Epoch [5/10], Loss: 1.3863
Epoch [6/10], Loss: 1.3863
Epoch [7/10], Loss: 1.3863
Epoch [8/10], Loss: 1.3863
Epoch [9/10], Loss: 1.3863
Epoch [10/10], Loss: 1.3863


In [46]:
def calculate_accuracy(model, X_test, y_test):
       with torch.no_grad():
           y_pred = model(X_test)
           _, predicted = torch.max(y_pred, 1)
           total = y_test.size(0)
           correct = (predicted == torch.argmax(y_test, dim=1)).sum().item()
           accuracy = 100 * correct / total
           return accuracy
accuracy = calculate_accuracy(model, X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 25.30%


## Exercise extra-credit: Make X and y a DataLoader, add batching, and validate the performance with the test set